# Lesson 6b: Factorization Machines with Keras

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/yb/mm0hrmlx3g9dv10c735753m00000gn/T/ipykernel_20805/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Okay bitte :((((

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from pathlib import Path
from zipfile import ZipFile
!mkdir models
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')
# complete_train = pd.read_csv('/content/drive/My Drive/_Universität_HPI/Semester 6/Recommenders/Assignments/Assignment 1/data/train.csv')
# complete_train_og = complete_train.copy()
# test = pd.read_csv('/content/drive/My Drive/_Universität_HPI/Semester 6/Recommenders/Assignments/Assignment 1/givenExample/kaggle_baseline.csv')
complete_train = pd.read_csv('train.csv')
complete_train_og = complete_train.copy()
test = pd.read_csv('kaggle_baseline.csv')

In [3]:
def text2seq(text, n_genre):
    """ using tokenizer to encoded the multi-level categorical feature
    """
    tokenizer = Tokenizer(lower=True, split='|',filters='', num_words=n_genre)
    tokenizer.fit_on_texts(text)
    seq = tokenizer.texts_to_sequences(text)
    seq = pad_sequences(seq, maxlen=3,padding='post')
    return seq

In [4]:
df = complete_train.copy()
df = df.rename(columns={"release_date": "movie_genre", "age": "sex", "sex": "age"})
df

,user_id,title,movie_id,rating,movie_genre,age,sex
0,2592,Top Gun (1986),1101,4,Action|Romance,50,M
1,4318,12 Angry Men (1957),1203,4,Drama,25,M
2,2756,Robocop 2 (1990),2986,2,Action|Crime|Sci-Fi,18,M
3,1706,Modern Times (1936),3462,5,Comedy,25,M
4,4813,Milk Money (1994),276,3,Comedy|Romance,35,F
...,...,...,...,...,...,...,...
800162,59,"Big Chill, The (1983)",2352,4,Comedy|Drama,50,F
800163,4458,So I Married an Axe Murderer (1993),543,4,Comedy|Romance|Thriller,25,F
800164,1234,Almost Famous (2000),3897,4,Comedy|Drama,18,M
800165,4864,"Fish Called Wanda, A (1988)",1079,5,Comedy,18,M


In [5]:
n_genre = 18
df['movie_genre'] = text2seq(df['movie_genre'].values, n_genre=n_genre).tolist()

In [ ]:
df

,user_id,title,movie_id,rating,movie_genre,age,sex
0,2592,Top Gun (1986),1101,4,"[3, 6, 0]",50,M
1,4318,12 Angry Men (1957),1203,4,"[2, 0, 0]",25,M
2,2756,Robocop 2 (1990),2986,2,"[3, 8, 5]",18,M
3,1706,Modern Times (1936),3462,5,"[1, 0, 0]",25,M
4,4813,Milk Money (1994),276,3,"[1, 6, 0]",35,F
...,...,...,...,...,...,...,...
800162,59,"Big Chill, The (1983)",2352,4,"[1, 2, 0]",50,F
800163,4458,So I Married an Axe Murderer (1993),543,4,"[1, 6, 4]",25,F
800164,1234,Almost Famous (2000),3897,4,"[1, 2, 0]",18,M
800165,4864,"Fish Called Wanda, A (1988)",1079,5,"[1, 0, 0]",18,M


In [6]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(df, test_size=0.2, random_state=7)

In [7]:
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *

def define_input_layers():
    # numerical features
    fea3_input = Input((1,), name = 'input_fea3')
    num_inputs = [fea3_input]
    # single level categorical features
    uid_input = Input((1,), name = 'input_uid') #user_id
    mid_input = Input((1,), name= 'input_mid')  #movie_id
    cat_sl_inputs = [uid_input, mid_input]

    # multi level categorical features (with 3 genres at most)
    genre_input = Input((3,), name = 'input_genre')
    cat_ml_inputs = [genre_input]

    inputs = num_inputs + cat_sl_inputs + cat_ml_inputs

    return inputs

inputs = define_input_layers()

In [8]:
def Tensor_Mean_Pooling(name = 'mean_pooling', keepdims = False):
    return Lambda(lambda x: K.mean(x, axis = 1, keepdims=keepdims), name = name)

def fm_1d(inputs, n_uid, n_mid, n_genre):

    # user feat3 + user embedding + movie embedding + genre embedding
    fea3_input, uid_input, mid_input, genre_input = inputs

    # all tensors are reshape to (None, 1)
    num_dense_1d = [Dense(1, name = 'num_dense_1d_fea4')(fea3_input)]
    cat_sl_embed_1d = [Embedding(n_uid + 1, 1, name = 'cat_embed_1d_uid')(uid_input),
                        Embedding(n_mid + 1, 1, name = 'cat_embed_1d_mid')(mid_input)]
    cat_ml_embed_1d = [Embedding(n_genre + 1, 1, mask_zero=True, name = 'cat_embed_1d_genre')(genre_input)]

    cat_sl_embed_1d = [Reshape((1,))(i) for i in cat_sl_embed_1d]
    cat_ml_embed_1d = [Tensor_Mean_Pooling(name = 'embed_1d_mean')(i) for i in cat_ml_embed_1d]

    # add all tensors
    y_fm_1d = Add(name = 'fm_1d_output')(num_dense_1d + cat_sl_embed_1d + cat_ml_embed_1d)

    return y_fm_1d

y_1d = fm_1d(inputs, 10, 10, 10)

/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/layer.py:845: UserWarning: Layer 'embed_1d_mean' (of type Lambda) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


In [10]:
def fm_2d(inputs, n_uid, n_mid, n_genre, k):

    fea3_input, uid_input, mid_input, genre_input = inputs

    num_dense_2d = [Dense(k, name = 'num_dense_2d_fea3')(fea3_input)] # shape (None, k)
    num_dense_2d = [Reshape((1,k))(i) for i in num_dense_2d] # shape (None, 1, k)

    cat_sl_embed_2d = [Embedding(n_uid + 1, k, name = 'cat_embed_2d_uid')(uid_input),
                       Embedding(n_mid + 1, k, name = 'cat_embed_2d_mid')(mid_input)] # shape (None, 1, k)

    cat_ml_embed_2d = [Embedding(n_genre + 1, k, name = 'cat_embed_2d_genre')(genre_input)] # shape (None, 3, k)
    cat_ml_embed_2d = [Tensor_Mean_Pooling(name = 'cat_embed_2d_genure_mean', keepdims=True)(i) for i in cat_ml_embed_2d] # shape (None, 1, k)

    # concatenate all 2d embed layers => (None, ?, k)
    embed_2d = Concatenate(axis=1, name = 'concat_embed_2d')(num_dense_2d + cat_sl_embed_2d + cat_ml_embed_2d)

    # calcuate the interactions by simplication
    # sum of (x1*x2) = sum of (0.5*[(xi)^2 - (xi^2)])
    tensor_sum = Lambda(lambda x: K.sum(x, axis = 1), name = 'sum_of_tensors')
    tensor_square = Lambda(lambda x: K.square(x), name = 'square_of_tensors')

    sum_of_embed = tensor_sum(embed_2d)
    square_of_embed = tensor_square(embed_2d)

    square_of_sum = Multiply()([sum_of_embed, sum_of_embed])
    sum_of_square = tensor_sum(square_of_embed)

    sub = Subtract()([square_of_sum, sum_of_square])
    sub = Lambda(lambda x: x*0.5)(sub)
    y_fm_2d = Reshape((1,), name = 'fm_2d_output')(tensor_sum(sub))

    return y_fm_2d, embed_2d

y_fm2_d, embed_2d = fm_2d(inputs, 10, 10, 10, 5)

In [11]:
def fm_model(n_uid, n_mid, n_genre, k, dnn_dr):

    inputs = define_input_layers()

    y_fm_1d = fm_1d(inputs, n_uid, n_mid, n_genre)
    y_fm_2d, embed_2d = fm_2d(inputs, n_uid, n_mid, n_genre, k)


    # combinded deep and fm parts
    y = Concatenate()([y_fm_1d, y_fm_2d])
    y = Dense(1, name = 'fm_output')(y)

    fm_model_1d = Model(inputs, y_fm_1d)
    fm_model_2d = Model(inputs, y_fm_2d)
    fm_model = Model(inputs, y)

    return fm_model_1d, fm_model_2d, fm_model

In [12]:
params = {
    'n_uid': df['user_id'].max(),
    'n_mid': df['movie_id'].max(),
    'n_genre': 18,
    'k':20,
    'dnn_dr': 0.5
}

fm_model_1d, fm_model_2d, fm_model = fm_model(**params)

/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/layer.py:845: UserWarning: Layer 'embed_1d_mean' (of type Lambda) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


In [13]:
def df2xy(ratings):
    x = [df['age'].values,
         df['user_id'].values,
         df['movie_id'].values,
         np.concatenate(df['movie_genre'].values).reshape(-1,3)]
    y = df['rating'].values
    return x,y

train_x, train_y = df2xy(train)
valid_x, valid_y = df2xy(val)

In [15]:
from tensorflow.keras.callbacks import  EarlyStopping, ModelCheckpoint
fm_model_2d.compile(
    loss=tf.keras.losses.MeanSquaredError(), optimizer=keras.optimizers.Adam(learning_rate=0.001)
)
early_stop = EarlyStopping(monitor='val_loss', patience=3)
model_ckp = ModelCheckpoint(filepath='./models/deepfm.weights.h5',
                            monitor='val_loss',
                            save_weights_only=True,
                            save_best_only=True)
callbacks = [model_ckp,early_stop]
train_history = fm_model_2d.fit(train_x, train_y,
                                  epochs=100, batch_size=2048,
                                  validation_data=(valid_x, valid_y),
                                  callbacks = callbacks)

Epoch 1/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 2.7122 - val_loss: 1.0261
Epoch 2/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1.0087 - val_loss: 0.9263
Epoch 3/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.9275 - val_loss: 0.8745
Epoch 4/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.8813 - val_loss: 0.8432
Epoch 5/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.8553 - val_loss: 0.8237
Epoch 6/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.8364 - val_loss: 0.8114
Epoch 7/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.8225 - val_loss: 0.8038
Epoch 8/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.8138 - val_loss: 0.7975
Epoch 9/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.8077 - val_loss: 0.7936
Epoch 10/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.8043 - val_loss: 0.7897
Epoch 11/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.7977 - val_loss: 0.7906
Epoch 12/100
391/391 ━━━━━━━━━━━━━━━━━━━━

In [ ]:

# train  model
fm_model.compile(
    loss=tf.keras.losses.MeanSquaredError(), optimizer=keras.optimizers.Adam(learning_rate=0.001)
)
early_stop = EarlyStopping(monitor='val_loss', patience=3)
model_ckp = ModelCheckpoint(filepath='./models/deepfm_weights.h5',
                            monitor='val_loss',
                            save_weights_only=True,
                            save_best_only=True)
callbacks = [model_ckp,early_stop]
train_history = fm_model.fit(train_x, train_y,
                                  epochs=100, batch_size=2048,
                                  validation_data=(valid_x, valid_y),
                                  callbacks = callbacks)

Epoch 1/100
391/391 [==============================] - 38s 91ms/step - loss: 0.4712 - val_loss: 0.4561
Epoch 2/100
391/391 [==============================] - 7s 17ms/step - loss: 0.4701 - val_loss: 0.4562
Epoch 3/100
391/391 [==============================] - 5s 14ms/step - loss: 0.4697 - val_loss: 0.4552
Epoch 4/100
391/391 [==============================] - 5s 13ms/step - loss: 0.4692 - val_loss: 0.4566
Epoch 5/100
391/391 [==============================] - 5s 13ms/step - loss: 0.4688 - val_loss: 0.4550
Epoch 6/100
391/391 [==============================] - 6s 16ms/step - loss: 0.4685 - val_loss: 0.4550
Epoch 7/100
391/391 [==============================] - 5s 12ms/step - loss: 0.4681 - val_loss: 0.4558
Epoch 8/100
391/391 [==============================] - 6s 15ms/step - loss: 0.4677 - val_loss: 0.4531
Epoch 9/100
391/391 [==============================] - 5s 13ms/step - loss: 0.4671 - val_loss: 0.4533
Epoch 10/100
391/391 [==============================] - 4s 11ms/step - loss: 0.46

0.4588

In [ ]:
fm_model_1d.compile(
    loss=tf.keras.losses.MeanSquaredError(), optimizer=keras.optimizers.Adam(learning_rate=0.001)
)
early_stop = EarlyStopping(monitor='val_loss', patience=3)
model_ckp = ModelCheckpoint(filepath='./models/deepfm_weights.h5',
                            monitor='val_loss',
                            save_weights_only=True,
                            save_best_only=True)
callbacks = [model_ckp,early_stop]
train_history = fm_model_1d.fit(train_x, train_y,
                                  epochs=100, batch_size=2048,
                                  validation_data=(valid_x, valid_y),
                                  callbacks = callbacks)

Epoch 1/30
391/391 [==============================] - 20s 49ms/step - loss: 140.3054 - val_loss: 43.0128
Epoch 2/30
391/391 [==============================] - 5s 12ms/step - loss: 17.0158 - val_loss: 5.3378
Epoch 3/30
391/391 [==============================] - 4s 9ms/step - loss: 3.8143 - val_loss: 3.2096
Epoch 4/30
391/391 [==============================] - 5s 13ms/step - loss: 3.0070 - val_loss: 2.8141
Epoch 5/30
391/391 [==============================] - 4s 10ms/step - loss: 2.6332 - val_loss: 2.4516
Epoch 6/30
391/391 [==============================] - 4s 10ms/step - loss: 2.2832 - val_loss: 2.1162
Epoch 7/30
391/391 [==============================] - 4s 11ms/step - loss: 1.9659 - val_loss: 1.8188
Epoch 8/30
391/391 [==============================] - 3s 7ms/step - loss: 1.6905 - val_loss: 1.5662
Epoch 9/30
391/391 [==============================] - 3s 7ms/step - loss: 1.4615 - val_loss: 1.3610
Epoch 10/30
391/391 [==============================] - 3s 9ms/step - loss: 1.2794 - val_l

Epoch 1/40
391/391 [==============================] - 19s 46ms/step - loss: 1.9383 - val_loss: 0.7073
Epoch 2/40
391/391 [==============================] - 5s 14ms/step - loss: 0.7165 - val_loss: 0.6988
Epoch 3/40
391/391 [==============================] - 5s 12ms/step - loss: 0.7111 - val_loss: 0.6936
Epoch 4/40
391/391 [==============================] - 5s 14ms/step - loss: 0.7065 - val_loss: 0.6897
Epoch 5/40
391/391 [==============================] - 4s 9ms/step - loss: 0.7016 - val_loss: 0.6848
Epoch 6/40
391/391 [==============================] - 3s 8ms/step - loss: 0.6968 - val_loss: 0.6793
Epoch 7/40
391/391 [==============================] - 4s 10ms/step - loss: 0.6918 - val_loss: 0.6745
Epoch 8/40
391/391 [==============================] - 4s 10ms/step - loss: 0.6867 - val_loss: 0.6707
Epoch 9/40
391/391 [==============================] - 4s 9ms/step - loss: 0.6821 - val_loss: 0.6644
Epoch 10/40
391/391 [==============================] - 4s 9ms/step - loss: 0.6766 - val_loss: